# Model Run 2

The goal here is to tweak the model a little bit to yield more localized strain.
Here again are the images from the previous notebook that show the results of other models, some of which are more similar to our results than others.
Think about what we can change in our model to get different results.

## Possible changes to make

Note that any change we make may also affect how long it takes to run the model. 
Our goal is to stay within the 10-minute time frame that the original model used.

Considerations
 - What type of plot is more helpful for visualizing strain localization?
 - Does this visualization actually need the full hour of model simulation time?
 - If we decrease the model simulation time from 1 hour to, say 30 minutes, what may change?

Here is a table of some things you may want to try and how they could effect the run time.

| Parameter | Change | Change in run time |
|-----------|-----------|----------------------|
| Mesh resolution | double it | ~4x longer     |
| Particles per cell | double it | ~2x longer  |
| Sand cohesion | reduce it | no effect? |
| Sand Friction angle | reduce it | no effect? |
| Sand Viscosity      | ??? | ??? |
| Solver tolerance    | increase by 10x | reduced but how much? |


Make your change in the code below, and run the cells to see the results. You can run the code cell-by-cell as you scroll down (Shift+Enter). If a cell fails to run, you probably have a typo.

## Imports and unit scaling
### Don't change this

In [ ]:
import UWGeodynamics as GEO
u = GEO.UnitRegistry

velocity = 2.5 * u.centimeter / u.hour
model_length = 20. * u.centimeter
bodyforce = 1560 * u.kilogram / u.metre**3 * 9.81 * u.meter / u.second**2

KL = model_length
Kt = KL / velocity
KM = bodyforce * KL**2 * Kt**2

GEO.scaling_coefficients["[length]"] = KL
GEO.scaling_coefficients["[time]"] = Kt
GEO.scaling_coefficients["[mass]"]= KM

In [ ]:
# General parameters
# Must be set before initializing the Model object

GEO.rcParams['nonlinear.tolerance'] = 1e-2 # default, 1e-2 (0.01)
GEO.rcParams['swarm.particles.per.cell.2D'] = 40 # default, 40

## Setup model resolution and domain size
### Change `elementRes` to increase resolution

In [4]:
Model = GEO.Model(elementRes=(120, 30), 
                  minCoord=(0. * u.centimeter, -3.5 * u.centimeter), 
                  maxCoord=(20. * u.centimeter, 1.5 * u.centimeter), 
                  gravity=(0.0, -9.81 * u.meter / u.second**2))

Model.mesh_advector(axis=0)

# This sets up a grid that is only used for visualization. Do not change.
Model.add_visugrid(elementRes=(120, 30), 
                   minCoord=(0. * u.centimeter, -3.5 * u.centimeter), 
                   maxCoord=(20. * u.centimeter, 0.0 * u.centimeter))

## Material layouts
### Do not change

In [8]:
air = Model.add_material(name="Air", shape=GEO.shapes.Layer(top=Model.top, bottom=Model.bottom))
sand1 = Model.add_material(name="Sand1", shape=GEO.shapes.Layer(top=0.*u.centimeter, bottom=Model.bottom))
sand2 = Model.add_material(name="Sand2", shape=GEO.shapes.Layer(top=-1. * u.centimeter, bottom=-2. * u.centimeter))

vertices = [( 5.* u.centimeter, -3.0 * u.centimeter),
            (15.* u.centimeter, -3.0 * u.centimeter),
            (15.* u.centimeter, -3.5 * u.centimeter),
            ( 5.* u.centimeter, -3.5 * u.centimeter)]

silicon = Model.add_material(name="Silicon", shape=GEO.shapes.Polygon(vertices))

## Passive Tracers
### No impact on model results

In [9]:
import numpy as np

npoints = 1000

coords = np.ndarray((npoints, 2))
coords[:, 0] = np.linspace(GEO.nd(Model.minCoord[0]), GEO.nd(Model.maxCoord[0]), npoints)
coords[:, 1] = GEO.nd(sand1.top)

Model.add_passive_tracers(name="Interface1", vertices=coords)

coords[:, 1] = GEO.nd(sand2.top)
Model.add_passive_tracers(name="Interface2", vertices=coords)

coords[:, 1] = GEO.nd(sand2.bottom)
Model.add_passive_tracers(name="Interface3", vertices=coords)

In [ ]:
from UWGeodynamics import visualisation as vis
Fig = vis.Figure(figsize=(1200,400))
Fig.Points(Model.Interface1_tracers, pointSize=2.0)
Fig.Points(Model.Interface2_tracers, pointSize=2.0)
Fig.Points(Model.Interface3_tracers, pointSize=2.0)
Fig.Points(Model.swarm, Model.materialField, fn_size=2.0)
Fig.Mesh(Model._visugrid.mesh, lineWidth=0.5)
Fig.show()

## Material densities and viscosities

In [12]:
# Define densities
air.density = 10. * u.kilogram / u.metre**3
sand1.density = 1560. * u.kilogram / u.metre**3
sand2.density = 1560. * u.kilogram / u.metre**3
silicon.density = 965. * u.kilogram / u.metre**3

# Scalar viscosities
air.viscosity     = 1.0e2  * u.pascal * u.second
sand1.viscosity   = 1.0e13 * u.pascal * u.second
sand2.viscosity   = 1.0e13 * u.pascal * u.second
silicon.viscosity = 5.0e4  * u.pascal * u.second

# Plasticity


In [14]:
sandPlasticity = GEO.DruckerPrager(cohesion=10. * u.pascal, # before softening
                                     cohesionAfterSoftening=10. * u.pascal,
                                     frictionCoefficient=0.73, # before softening, radians
                                     frictionAfterSoftening=0.60) # after softening, radians

# note that 30 degress ~ 0.5 rad

# assign plastic behaviour to sand1 and sand2 materials
sand1.plasticity = sandPlasticity
sand2.plasticity = sandPlasticity

## Boundary conditions
### Do not change

In [ ]:
import underworld.function as fn

conditions = [(Model.x > GEO.nd(10.1 * u.centimetre), GEO.nd(2.5 * u.centimeter / u.hour)),
              (Model.x > GEO.nd(9.9 * u.centimetre), (Model.x - GEO.nd(9.9 * u.centimetre)) * GEO.nd(velocity) / GEO.nd(0.2 * u.centimetre)),
              (True, 0.0)]

fn_condition = fn.branching.conditional(conditions)

Model.set_velocityBCs(left=[0 * u.centimeter / u.hour, None],
                      right=[2.5 * u.centimeter / u.hour, None],
                      bottom=[fn_condition, 0.0])

## Initialise Model

In [17]:
Model.minViscosity = 1.0e2 * u.pascal * u.second
Model.maxViscosity = 1.0e9 * u.pascal * u.second

Model.init_model(pressure="lithostatic")

In [ ]:
FigPressure = vis.Figure(figsize=(1200, 400), title="Pressure Field (Pa)", quality=3)
FigPressure.Surface(Model.mesh, GEO.dimensionalise(Model.pressureField, u.pascal))
FigPressure.show()

In [ ]:
FigViscosity = vis.Figure(figsize=(1200, 400), title="Viscosity Field (Pa.s)", quality=3)
FigViscosity.Points(Model.swarm, GEO.dimensionalise(Model.viscosityField, u.pascal * u.second), pointSize=2.0)
FigViscosity.show()

## Run model

In [23]:
Model.solver.set_inner_method("mumps")
Model.solver.set_penalty(1e6)

Model.run_for(1.0 * u.hours) # run 1 was 1.0 * u.hours

In [ ]:
# Show the figure of material layouts

Fig.show()

In [ ]:
# Create figure for strain rate

fn = GEO.uw.function

map_dict = {
    air.index: 0.,
    sand1.index: 1.,
    sand2.index: 1.,
    silicon.index: 1.
}
mask = fn.branching.map(fn_key=Model.materialField, mapping=map_dict)

FigStrainRate = vis.Figure(figsize=(1200, 400), title="Strain Rate", quality=3)
FigStrainRate.Points(Model.swarm, fn_colour=Model.strainRateField, fn_mask=mask, pointSize=1.5, logScale=True, colours="darkblue blue cyan green yellow orange red darkred")
FigStrainRate.Points(Model.Interface1_tracers)
FigStrainRate.Points(Model.Interface2_tracers)
FigStrainRate.Points(Model.Interface3_tracers)
FigStrainRate.show()